# Problem 4

In this problem, we simply finetune a BERT model (not pretrained) on RTE dataset, and then finetune a BERT model (pretrained) on RTE dataset.

**IMPORTANT NOTES**:
- Please make sure that you have already read the part of hw5 pdf that corresponds to this problem. This is very important.
- At the end of the hw5, you will need to submit a zip folder containing three things. The instruction is also included in the first paragraph of the hw5 pdf.
  - (1) The writeup pdf containing your solutions to Problems 1, 2, 3, 4, 5. Yes, there're things you need to respond in your writeup (see hw5 pdf).
  - (2) The downloaded colab corresponding to Problem 4.
  - (3) The downloaded colab corresponding to Problem 5.

Some imports and data downloading

In [1]:
import pandas as pd

In [2]:
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, Optional

import numpy as np
import pandas as pd
import torch
import torch.nn as nn 
from transformers import AutoTokenizer, BertConfig, BertForSequenceClassification, BertTokenizer,EvalPrediction, GlueDataset, GlueDataTrainingArguments, AutoModel, BertPreTrainedModel, AutoConfig, BertModel
from transformers import GlueDataTrainingArguments 
from transformers import (
    Trainer,
    TrainingArguments,
    glue_compute_metrics,
    glue_tasks_num_labels,
    set_seed,
)

import warnings
warnings.filterwarnings('ignore')

In [3]:
#!git clone https://github.com/huggingface/transformers
#!python transformers/utils/download_glue_data.py --tasks RTE
#!pip install transformers

In [4]:
df=pd.read_csv('SST/train.tsv', sep='\t')

In [5]:
df.head()

,sentence,label
0,hide new secretions from the parental units,0
1,"contains no wit , only labored gags",0
2,that loves its characters and communicates som...,1
3,remains utterly satisfied to remain the same t...,0
4,on the worst revenge-of-the-nerds clichés the ...,0


### From non-pretrained BERT

In [6]:
class SequenceClassificationBERT(nn.Module):
      
    def __init__(self, config, bert_model, weights = [0.5,0.5]):
        super().__init__()
        self.config = config
        self.num_labels = config.num_labels
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.bert = bert_model
        self.weights = weights

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # make sure that all the arguments in the forward() function is used
        # somewhere in the code

        ##### 

        # TODO

        #####

        outputs = self.bert(input_ids=input_ids, 
                               attention_mask=attention_mask, 
                               token_type_ids=token_type_ids,
                               position_ids=position_ids,
                               head_mask=head_mask,
                               inputs_embeds=inputs_embeds, 
                               output_attentions=output_attentions,
                               output_hidden_states=output_hidden_states,
                               return_dict=return_dict) 

        pooled_out = outputs[1] 
        pooled_out = self.dropout(pooled_out)
        #print(pooled_out.shape)
        #h_cls = pooled_out[:, 0]
        #print(h_cls.shape)
        logits = self.classifier(pooled_out)

        # RTE (2 classes)
        loss_fct = nn.CrossEntropyLoss(torch.tensor(self.weights, device='cuda'))
        loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1)) 

        # do not change the lines below, so make sure your code works for the
        # lines below

        output = (logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output


In [7]:
def compute_metrics_pretrained(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return glue_compute_metrics(data_args.task_name, preds, p.label_ids)

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions[0], axis=1) #predictions
    return glue_compute_metrics(data_args.task_name, preds, p.label_ids)

In [8]:
def non_pretrained(weights):
    set_seed(42)
    num_labels = glue_tasks_num_labels[data_args.task_name]

    tokenizer = BertTokenizer.from_pretrained(model_name)
    config = BertConfig.from_pretrained(model_name, output_hidden_states=False, output_attentions=True)
    bert_model = AutoModel.from_config(config)
    # Loading a model from its configuration file does not load the model weights. 
    # It only affects the model’s configuration. Use from_pretrained() to load the model weights.

    model = SequenceClassificationBERT(config=config, bert_model=bert_model,weights=weights)

    train_dataset = GlueDataset(data_args, tokenizer=tokenizer)
    eval_dataset = GlueDataset(data_args, tokenizer=tokenizer, mode="dev")

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )



    trainer.train()
    trainer.evaluate()
    
    d=trainer.evaluate()
    return d['eval_acc'], d['eval_loss']

### From pretrained BERT

In [9]:
def pretrained(weights):
    set_seed(42)
    tokenizer = BertTokenizer.from_pretrained(model_name)
    config = BertConfig.from_pretrained(model_name, output_hidden_states=False, output_attentions=True)
    bert_model = AutoModel.from_pretrained(model_name, config)

    train_dataset = GlueDataset(data_args, tokenizer=tokenizer)
    eval_dataset = GlueDataset(data_args, tokenizer=tokenizer, mode="dev")

    model = SequenceClassificationBERT(config=config, bert_model=bert_model,weights=weights)


    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics_pretrained,
    )

    trainer.train()
    d=trainer.evaluate()
    return d['eval_acc'], d['eval_loss']

In [10]:
non_pre_acc=[]
pre_acc=[]
non_pre_loss=[]
pre_loss=[]
for i in [0.1, 0.25, 0.5, 0.75, 1]:
    train = pd.read_csv('SST/train.tsv', sep='\t')
    dev = pd.read_csv('SST/dev.tsv', sep='\t')
    test = pd.read_csv('SST/test.tsv', sep='\t')
    
    df1= train[train.label==1].sample(frac=i, random_state=1).reset_index(drop=True)
    print(train.label.value_counts())
    print(df1.shape)
    
    df2= train[train.label==0].reset_index(drop=True)
    frames=[df1,df2]
    df=pd.concat(frames).reset_index(drop=True)
    #df[df['label'] == 0]['label'] = 'neg'
    #df[df['label'] == 1]['label'] = 'pos'
    #print(df.head())
    #df
    #dev[dev['label'] == 0]['label'] = 'neg'
    #dev[dev['label'] == 1]['label'] = 'pos'
    
    df.to_csv('SST_NEW/train.tsv', sep='\t',index=False)
    dev.to_csv('SST_NEW/dev.tsv', sep='\t',index=False)
    test.to_csv('SST_NEW/test.tsv', sep='\t',index=False)
    
    model_name = "bert-base-cased"

    data_args = GlueDataTrainingArguments(task_name="sst-2", data_dir="./SST_NEW")

    training_args = TrainingArguments(
        logging_steps=50, 
        per_device_train_batch_size=32, 
        per_device_eval_batch_size=64, 
        save_steps=1000,
        evaluation_strategy='epoch',
        output_dir="./models/sst-2",
        overwrite_output_dir=True,
        do_train=True,
        do_eval=True,
        do_predict=True,
        learning_rate=0.00001,
        num_train_epochs=15,
    )
    weights = torch.tensor([1,1/i])
    print('Fraction ', i)
    print('Non Pretrained ')
    acc1, loss1= non_pretrained(weights)
    print('Pretrained')
    acc2, loss2= pretrained(weights)
    non_pre_acc.append(acc1)
    pre_acc.append(acc2)
    non_pre_loss.append(loss1)
    pre_loss.append(loss2)
    


1    37569
0    29780
Name: label, dtype: int64
(3757, 2)
Fraction  0.1
Non Pretrained 


Epoch,Training Loss,Validation Loss,Acc,Runtime,Samples Per Second
1,0.564700,0.509555,0.727064,11.424500,76.327000
2,0.449300,0.871545,0.730505,11.172200,78.051000
3,0.296300,0.683795,0.763761,11.172000,78.052000
4,0.343000,1.246229,0.692661,11.063100,78.820000
5,0.290700,0.998677,0.728211,11.159300,78.141000
6,0.278700,1.231931,0.732798,11.149400,78.211000
7,0.261700,0.829916,0.766055,11.102600,78.540000
8,0.241600,1.457525,0.735092,11.154500,78.175000
9,0.218600,1.351178,0.774083,11.231300,77.640000
10,0.190100,1.873821,0.737385,11.192700,77.908000


Pretrained


Epoch,Training Loss,Validation Loss,Acc,Runtime,Samples Per Second
1,0.290400,0.475874,0.876147,2.239000,389.456000
2,0.300600,0.652426,0.878440,2.229700,391.081000
3,0.160200,0.558211,0.893349,2.246100,388.226000
4,0.175500,0.803028,0.891055,2.266500,384.733000
5,0.053100,0.553898,0.910550,2.254900,386.717000
6,0.063000,1.102687,0.870413,2.242800,388.800000
7,0.056000,0.921723,0.900229,2.237200,389.776000
8,0.008000,1.141868,0.896789,2.234300,390.281000
9,0.029100,1.212320,0.899083,2.247500,387.981000
10,0.057600,1.545506,0.883028,2.260700,385.720000


1    37569
0    29780
Name: label, dtype: int64
(9392, 2)
Fraction  0.25
Non Pretrained 


...

Epoch,Training Loss,Validation Loss,Acc,Runtime,Samples Per Second
1,0.169300,0.475354,0.866972,2.256100,386.507000
2,0.122200,0.690083,0.848624,2.265500,384.912000
3,0.072400,0.727412,0.848624,2.268100,384.456000
4,0.052400,0.551563,0.893349,2.271000,383.971000
5,0.034900,0.530743,0.900229,2.270900,383.983000
6,0.033900,0.729244,0.881881,2.273500,383.556000
7,0.017800,0.593829,0.901376,2.263400,385.266000
8,0.008200,0.969160,0.879587,2.268200,384.447000
9,0.006600,1.022600,0.881881,2.263800,385.186000
10,0.005100,1.031071,0.886468,2.260600,385.744000


1    37569
0    29780
Name: label, dtype: int64
(37569, 2)
Fraction  1
Non Pretrained 


Epoch,Training Loss,Validation Loss,Acc,Runtime,Samples Per Second
1,0.292300,0.949649,0.588303,11.451500,76.147000
2,0.218800,1.052260,0.635321,11.009700,79.203000
3,0.176500,0.766539,0.701835,11.127600,78.363000
4,0.176400,1.123114,0.701835,11.118300,78.429000
5,0.145300,0.897699,0.683486,11.148700,78.215000
6,0.129800,1.241023,0.672018,11.105100,78.522000
7,0.123300,0.925030,0.724771,11.204900,77.823000
8,0.105800,1.602572,0.678899,11.204200,77.828000
9,0.104300,1.207148,0.725917,11.137800,78.292000
10,0.090000,1.109685,0.747706,11.103600,78.533000


Pretrained


Epoch,Training Loss,Validation Loss,Acc,Runtime,Samples Per Second
1,0.144800,0.412951,0.870413,2.257700,386.234000
2,0.103500,0.597958,0.853211,2.269300,384.266000
3,0.062000,0.523389,0.860092,2.258800,386.040000
4,0.047500,0.559671,0.883028,2.259600,385.915000
5,0.040900,0.442750,0.895642,2.257300,386.305000
6,0.031500,0.687456,0.878440,2.266200,384.782000
7,0.012800,0.655201,0.895642,2.261100,385.657000
8,0.013100,0.773781,0.888761,2.258400,386.121000
9,0.008400,0.916159,0.879587,2.258600,386.072000
10,0.004600,0.878264,0.897936,2.259300,385.968000


In [11]:
print(non_pre_acc)
print(pre_acc)
print(non_pre_loss)
print(pre_loss)

[0.7408256880733946, 0.7408256880733946, 0.7327981651376146, 0.7282110091743119, 0.7178899082568807]
[0.8967889908256881, 0.8807339449541285, 0.8830275229357798, 0.8807339449541285, 0.8818807339449541]
[2.3670077323913574, 2.324096202850342, 2.0706653594970703, 1.841522455215454, 1.651656150817871]
[1.5855176448822021, 1.624110460281372, 1.3880287408828735, 1.2626886367797852, 1.1161582469940186]


non_pre_acc=[]
pre_acc=[]
non_pre_loss=[]
pre_loss=[]
for i in [0.1, 0.25, 0.5, 0.75, 1]:
    train = pd.read_csv('SST/train.tsv', sep='\t')
    dev = pd.read_csv('SST/dev.tsv', sep='\t')
    test = pd.read_csv('SST/test.tsv', sep='\t')
    
    df1= train[train.label==0].sample(frac=i, random_state=1).reset_index(drop=True)
    print(train.label.value_counts())
    print(df1.shape)
    
    df2= train[train.label==1].reset_index(drop=True)
    frames=[df1,df2]
    df=pd.concat(frames).reset_index(drop=True)
    #df[df['label'] == 0]['label'] = 'neg'
    #df[df['label'] == 1]['label'] = 'pos'
    #print(df.head())
    #df
    #dev[dev['label'] == 0]['label'] = 'neg'
    #dev[dev['label'] == 1]['label'] = 'pos'
    
    df.to_csv('SST_NEW/train.tsv', sep='\t',index=False)
    dev.to_csv('SST_NEW/dev.tsv', sep='\t',index=False)
    test.to_csv('SST_NEW/test.tsv', sep='\t',index=False)
    
    model_name = "bert-base-cased"

    data_args = GlueDataTrainingArguments(task_name="sst-2", data_dir="./SST_NEW")

    training_args = TrainingArguments(
        logging_steps=50, 
        per_device_train_batch_size=32, 
        per_device_eval_batch_size=64, 
        save_steps=1000,
        evaluation_strategy='epoch',
        output_dir="./models/sst-2",
        overwrite_output_dir=True,
        do_train=True,
        do_eval=True,
        do_predict=True,
        learning_rate=0.00001,
        num_train_epochs=15,
    )
    weights = torch.tensor([1/i,1])
    print('Fraction ', i)
    print('Non Pretrained ')
    acc1, loss1= non_pretrained(weights)
    print('Pretrained')
    acc2, loss2= pretrained(weights)
    non_pre_acc.append(acc1)
    pre_acc.append(acc2)
    non_pre_loss.append(loss1)
    pre_loss.append(loss2)
    


print(non_pre_acc1)
print(pre_acc1)
print(non_pre_loss1)
print(pre_loss1)